In [1]:
import os 
import sys

import FINE as fn

cwd = os.getcwd()

%load_ext autoreload
%autoreload 2

# Workflow for spatial aggregation of an energy system model

The data contained within an Energy System Model (ESM) instance is vast and complex. Saving it directly is not possible. It can, however, be saved as a NetCDF file which supports complex data structures. 

#### What exactly is NetCDF? 
NetCDF (Network Common Data Format) is a set of software libraries and machine-independent data formats that support the creation, access, and sharing of array-oriented scientific data. It is also a community standard for sharing scientific data. 

#### Python modules that support working with NetCDF files:
1. netcdf4-python: Official Python interface to netCDF files
2. PyNIO: To access different file formats such as netCDF, HDF, and GRIB
3. xarray: Based on NumPy and pandas

Note: xarray module is used here. 

For our use case, the following functionalities are provided: 
* Conversion of ESM instance to xarray dataset. Additionally, possible to save this dataset as NetCDF file in a desired folder, with a desired file name. 
* Conversion of xarray dataset/saved NetCDF file back to ESM instance.

#### Structure of the xarray dataset: 

<img src="xarray_fine.png" style="width: 1000px;"/>


## STEP 1. Set up your ESM instance 

In [2]:
sys.path.append(os.path.join(cwd, '..', 'Multi-regional_Energy_System_Workflow'))
from getData import getData

data = getData()

# 1. Create an energy system model instance
locations = {'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'}
commodityUnitDict = {'electricity': r'GW$_{el}$', 'methane': r'GW$_{CH_{4},LHV}$', 'biogas': r'GW$_{biogas,LHV}$',
                     'CO2': r'Mio. t$_{CO_2}$/h', 'hydrogen': r'GW$_{H_{2},LHV}$'}
commodities = {'electricity', 'hydrogen', 'methane', 'biogas', 'CO2'}

esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760,
                           commodityUnitsDict=commodityUnitDict,
                           hoursPerTimeStep=1, costUnit='1e9 Euro', lengthUnit='km', verboseLogLevel=0)

CO2_reductionTarget = 1


# 2. Add commodity sources to the energy system model
### Wind onshore

esM.add(fn.Source(esM=esM, name='Wind (onshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (onshore), operationRateMax'],
                  capacityMax=data['Wind (onshore), capacityMax'],
                  investPerCapacity=1.1, opexPerCapacity=1.1*0.02, interestRate=0.08,
                  economicLifetime=20))

### PV

esM.add(fn.Source(esM=esM, name='PV', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['PV, operationRateMax'], capacityMax=data['PV, capacityMax'],
                  investPerCapacity=0.65, opexPerCapacity=0.65*0.02, interestRate=0.08,
                  economicLifetime=25))


# 3. Add conversion components to the energy system model

### New combined cycly gas turbines for hydrogen
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (hydrogen)', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True,
                      investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### Electrolyzers
esM.add(fn.Conversion(esM=esM, name='Electroylzers', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True,
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.08,
                      economicLifetime=10))


# 4. Add commodity storages to the energy system model

### Lithium ion batteries
esM.add(fn.Storage(esM=esM, name='Li-ion batteries', commodity='electricity',
                   hasCapacityVariable=True, chargeEfficiency=0.95,
                   cyclicLifetime=10000, dischargeEfficiency=0.95, selfDischarge=1-(1-0.03)**(1/(30*24)),
                   chargeRate=1, dischargeRate=1, doPreciseTsaModeling=False,
                   investPerCapacity=0.151, opexPerCapacity=0.002, interestRate=0.08,
                   economicLifetime=22))

### Hydrogen filled salt caverns
esM.add(fn.Storage(esM=esM, name='Salt caverns (hydrogen)', commodity='hydrogen',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=133,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (hydrogen), capacityMax'],
                   investPerCapacity=0.00011, opexPerCapacity=0.00057, interestRate=0.08,
                   economicLifetime=30))


# 5. Add commodity transmission components to the energy system model

### AC cables
esM.add(fn.LinearOptimalPowerFlow(esM=esM, name='AC cables', commodity='electricity',
                                  hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix'],
                                  reactances=data['AC cables, reactances']))

### DC cables
esM.add(fn.Transmission(esM=esM, name='DC cables', commodity='electricity', losses=data['DC cables, losses'],
                        distances=data['DC cables, distances'],
                        hasCapacityVariable=True, capacityFix=data['DC cables, capacityFix']))


### Hydrogen pipelines
esM.add(fn.Transmission(esM=esM, name='Pipelines (hydrogen)', commodity='hydrogen',
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=False, bigM=300,
                        locationalEligibility=data['Pipelines, eligibility'],
                        capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                        investPerCapacity=0.000177, investIfBuilt=0.00033,
                        interestRate=0.08, economicLifetime=40))

# 6. Add commodity sinks to the energy system model

### Electricity demand
esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                hasCapacityVariable=False, operationRateFix=data['Electricity demand, operationRateFix']))

## 7.2. Hydrogen sinks
FCEV_penetration=0.5
esM.add(fn.Sink(esM=esM, name='Hydrogen demand', commodity='hydrogen', hasCapacityVariable=False,
                operationRateFix=data['Hydrogen demand, operationRateFix']*FCEV_penetration))


The distances of a component are set to a normalized value of 1.


 c:\users\s.patil\documents\code\fine\FINE\utils.py:673: UserWarning:A declaration of bigM is not necessary if hasIsBuiltBinaryVariable is set to false. The value of bigM will be ignored in the optimization.


## STEP 2. Spatial grouping of regions

In [7]:
SHAPEFILE_PATH = os.path.join(cwd, '..', 'Multi-regional_Energy_System_Workflow/InputData/SpatialData/ShapeFiles/clusteredRegions.shp')

In [4]:
sds_region_filename='aggregated_regions.shp'
sds_xr_dataset_filename='aggregated_xr_ds.nc4'

In [5]:
aggregation_function_dict = {'operationRateMax': ('mean', None),
                             'operationRateFix': ('sum', None),
                             'locationalEligibility': ('bool', None),
                             'capacityMax': ('sum', None),
                             'investPerCapacity': ('sum', None),
                             'investIfBuilt': ('sum', None),
                             'opexPerOperation': ('sum', None),
                             'opexPerCapacity': ('sum', None),
                             'opexIfBuilt': ('sum', None),
                             'interestRate': ('mean', None),
                             'economicLifetime': ('mean', None),
                             'capacityFix': ('sum', None),
                             'losses': ('mean', None),
                             'distances': ('mean', None),
                             'commodityCost': ('mean', None),
                             'commodityRevenue': ('mean', None),
                             'opexPerChargeOperation': ('mean', None),
                             'opexPerDischargeOperation': ('mean', None),
                             'QPcostScale': ('sum', None), 
                              'technicalLifetime': ('sum', None)}

In [8]:
aggregated_esM = esM.aggregateSpatially(shapefilePath=SHAPEFILE_PATH, 
                                       grouping_mode='parameter_based', 
                                       nRegionsForRepresentation=3,
                                       aggregatedResultsPath='output_data', 
                                       aggregation_function_dict=aggregation_function_dict,
                                       sds_region_filename=sds_region_filename,
                                       sds_xr_dataset_filename=sds_xr_dataset_filename)

 c:\users\s.patil\documents\code\fine\FINE\utils.py:1099: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1099: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1099: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1099: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


Performing parameter-based grouping on the regions.


 c:\users\s.patil\documents\code\fine\FINE\spagat\grouping_utils.py:211: UserWarning:Variable 0d_name has dimensions ('component',) which are not considered for spatial aggregation.
 c:\users\s.patil\documents\code\fine\FINE\spagat\grouping_utils.py:211: UserWarning:Variable 0d_commodity has dimensions ('component',) which are not considered for spatial aggregation.
 c:\users\s.patil\documents\code\fine\FINE\spagat\grouping_utils.py:211: UserWarning:Variable 0d_hasCapacityVariable has dimensions ('component',) which are not considered for spatial aggregation.
 c:\users\s.patil\documents\code\fine\FINE\spagat\grouping_utils.py:211: UserWarning:Variable 0d_capacityVariableDomain has dimensions ('component',) which are not considered for spatial aggregation.
 c:\users\s.patil\documents\code\fine\FINE\spagat\grouping_utils.py:211: UserWarning:Variable 0d_capacityPerPlantUnit has dimensions ('component',) which are not considered for spatial aggregation.
 c:\users\s.patil\documents\code\fin

The cophenetic correlation coefficient of the hiearchical clustering is  0.5610418835904831
Inconsistencies: [0.0, 0.7071067811865475, 0.0, 0.7071067811865472, 0.7071067811865474, 1.1272308730184022, 0.7071067811865452]
Silhouette scores:  [0.054270202828483674, 0.30351294763901604, 0.1678556280347441, 0.3996558757034347, 0.2155832157551782, 0.15188589319266743]
elapsed time for perform_parameter_based_grouping: 0.04 minutes
aggregation_function_dict found in kwargs


 c:\users\s.patil\documents\code\fine\FINE\spagat\representation.py:120: VisibleDeprecationWarning:Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1099: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1099: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1099: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:

TypeError: Invalid value for attr: {'cluster_0_cluster_2', 'cluster_7', 'cluster_1_cluster_3_cluster_4_cluster_5_cluster_6'} must be a number, a string, an ndarray or a list/tuple of numbers/strings for serialization to netCDF files

In [ ]:
new_locations = list(aggregated_esM.locations)
new_locations

In [ ]:
aggregated_esM.componentModelingDict['TransmissionModel'].componentsDict

# STEP 3. VRES Representation (Optional)

In [ ]:
## DATA 
ONSHORE_WIND_DATA_PATH = 'InputData/RERepresentationData/DEU_wind.nc4'
PV_DATA_PATH = 'InputData/RERepresentationData/DEU_PV.nc4'

#SHAPEFILE WITH MERGED REGIONS 
SHP_PATH = f'OutputData/{sds_region_filename}'

In [ ]:
## Representation 
represented_wind_ds = represent_RE_technology(ONSHORE_WIND_DATA_PATH,
                                              SHP_PATH, 
                                              n_timeSeries_perRegion=5,
                                              index_col='space', 
                                              geometry_col='geometry',
                                              longitude='x', 
                                              latitude='y') 

represented_pv_ds = represent_RE_technology(PV_DATA_PATH, 
                                            SHP_PATH, 
                                            n_timeSeries_perRegion=5,
                                            index_col='space', 
                                            geometry_col='geometry',
                                            longitude='x', 
                                            latitude='y')

In [ ]:
represented_wind_ds

In [ ]:
## Now we need to delete 'Wind (onshore)' and 'PV'in aggregated_esM and add the represented results 
aggregated_esM.componentModelingDict['SourceSinkModel'].componentsDict

In [ ]:
## But first we need certain info corresponding to these techs as they remain the same:
wind_investPerCapacity = aggregated_esM.getComponentAttribute('Wind (onshore)', 'investPerCapacity').mean()
wind_opexPerCapacity = aggregated_esM.getComponentAttribute('Wind (onshore)', 'opexPerCapacity').mean()
wind_interestRate = aggregated_esM.getComponentAttribute('Wind (onshore)', 'interestRate').mean()
wind_economicLifetime = aggregated_esM.getComponentAttribute('Wind (onshore)', 'economicLifetime').mean()

pv_investPerCapacity = aggregated_esM.getComponentAttribute('PV', 'investPerCapacity').mean()
pv_opexPerCapacity = aggregated_esM.getComponentAttribute('PV', 'opexPerCapacity').mean()
pv_interestRate = aggregated_esM.getComponentAttribute('PV', 'interestRate').mean()
pv_economicLifetime = aggregated_esM.getComponentAttribute('PV', 'economicLifetime').mean()

In [ ]:
## And now we delete them
aggregated_esM.removeComponent('Wind (onshore)')
aggregated_esM.removeComponent('PV')

In [ ]:
aggregated_esM.componentModelingDict['SourceSinkModel'].componentsDict

In [ ]:
## Prepare the representation results and add them to aggregated_esM
data = {}   

time_steps = aggregated_esM.totalTimeSteps
regions = represented_wind_ds['region_ids'].values
clusters = represented_wind_ds['TS_ids'].values  #clusters per region (also the locations for optimization)


for i, cluster in enumerate(clusters):
    # Add a wind turbine
    data.update({f'Wind (onshore), capacityMax {i}': pd.Series(represented_wind_ds.capacity.loc[:,cluster], index=regions)})

    data.update({f'Wind (onshore), operationRateMax {i}': pd.DataFrame(represented_wind_ds.capfac.loc[:,:,cluster].values,
                                                                       index=time_steps, columns=regions)})
    

    # Add a pv
    data.update({f'PV, capacityMax {i}': pd.Series(represented_pv_ds.capacity.loc[:,cluster], index=regions)})

    data.update({f'PV, operationRateMax {i}': pd.DataFrame(represented_pv_ds.capfac.loc[:,:,cluster].values,
                                                                       index=time_steps, columns=regions)})

In [ ]:
for i, cluster in enumerate(clusters):
    aggregated_esM.add(fn.Source(esM=aggregated_esM, 
                      name=f'Wind (onshore) {i}',
                      commodity='electricity', 
                      hasCapacityVariable=True,
                      operationRateMax=data[f'Wind (onshore), operationRateMax {i}'],
                      capacityMax=data[f'Wind (onshore), capacityMax {i}'],
                      investPerCapacity=wind_investPerCapacity, 
                      opexPerCapacity=wind_opexPerCapacity,
                      interestRate=pv_interestRate,
                      economicLifetime=wind_economicLifetime
                      ))
    
    aggregated_esM.add(fn.Source(esM=aggregated_esM, 
                      name=f'PV {i}', 
                      commodity='electricity',
                      hasCapacityVariable=True,
                      operationRateMax=data[f'PV, operationRateMax {i}'], 
                      capacityMax=data[f'PV, capacityMax {i}'],
                      investPerCapacity=pv_investPerCapacity, 
                      opexPerCapacity=pv_opexPerCapacity, 
                      interestRate=pv_interestRate,
                      economicLifetime=pv_economicLifetime))

In [ ]:
aggregated_esM.componentModelingDict['SourceSinkModel'].componentsDict

# 8.2 Temporal Aggregation

All components are now added to the model and the model can be optimized. If the computational complexity of the optimization should be reduced, the time series data of the specified components can be clustered before the optimization and the parameter timeSeriesAggregation is set to True in the optimize call.

In [ ]:
aggregated_esM.cluster(numberOfTypicalPeriods=7)

# 9 Optimization

In [ ]:
aggregated_esM.optimize(timeSeriesAggregation=True, solver='gurobi')

In [ ]:
# 10. Selected Results Output

In [ ]:
## 10.1. Plots for Spatial Aggregation Results 

In [ ]:
#### Original spatial resolution

In [ ]:
locFilePath = os.path.join(cwd, 'InputData', 'SpatialData','ShapeFiles', 'clusteredRegions.shp')
fig, ax = fn.plotLocations(locFilePath, plotLocNames=True, indexColumn='index')

In [ ]:
#### Spatial resolution after aggregation

In [ ]:
aggregated_regions_FilePath = os.path.join(cwd, 'OutputData', 'aggregated_regions.shp')
fig, ax = fn.plotLocations(aggregated_regions_FilePath, plotLocNames=True, indexColumn='space')

In [ ]:
## 10.2. Sources and Sink Optimization Summary